# Notebook
## Sistem Rekomendasi Obat berdasarkan _Weighted Hybrid Approach_

Notebook ini menyajikan penerapan dan eksperimentasi _weighted hybrid approach_ pada sistem rekomendasi obat. _Weighted hybrid approach_ yang diusulkan merupakan kombinasi antara _content-based filtering_  dan _collaborative filtering_. Beberapa teknik pembobotan yang akan dimasukkan dalam eksperimentasi antara lain _optimal weight_.

## 1. Pustaka

Impor pustaka yang dibutuhkan.

In [1]:
# pustaka eksternal
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist, squareform
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV

# pustaka custom
from analysis import *

## 2. Persiapan Set Data

Pada bagian ini, dilakukan persiapan set data yang dibutuhkan dalam penelitian. Karena set data yang sesuai untuk secara langsung digunakan dalam penelitian ini belum tersedia saat penelitian ini dikerjakan, maka Peneliti melakukan pengumpulan data dari beberapa sumber dan menggabungkannya.

Set data `WebMD` dan `DailyMed` digabungkan dalam struktur data DataFrame.

In [2]:
# berkas WebMD Drug Reviews oleh Rohan Harode dari Kaggle
path_wmd = "data/webmd/webmd.csv"

# berkas DailyMed Drug dari situs web DailyMed
# yang dikompilasi dengan menggunakan generate_dailymed.py
path_dm = "data/dailymed/dailymed.csv"

df_wmd = pd.read_csv(path_wmd)
df_dm = pd.read_csv(path_dm)

Atribut yang diperlukan sebagai pelengkap data pada DailyMed, antara lain:
1. nama obat secara umum pada kolom `sub_name`
2. daftar bahan aktif pada kolom `list_activeIngredient`, dan 
3. daftar bahan inaktif pada kolom `list_activeIngredient`.

Oleh karena itu, perlu dilakukan seleksi kolom pada DataFrame `df_dm` sebagai berikut.

In [3]:
df_dm = df_dm[["SubName", "ListActiveIngredient", "ListInactiveIngredient"]]

## 3. Prapemrosesan
Tahapan prapemrosesan data dilakukan untuk meningkatkan kualitas data yang meliputi data preparation, selection, data cleaning dan data tranformation. 

### 3.1. Data Integration

Tahap pertama pada data integration yaitu melakukan integrasi set data DailyMed dan WebMD pada _df_drugs_

In [4]:
df_drugs = pd.merge(df_dm, df_wmd, left_on = "SubName", right_on = "Drug")

In [5]:
df_drugs.columns

Index(['SubName', 'ListActiveIngredient', 'ListInactiveIngredient', 'Age',
       'Condition', 'Date', 'Drug', 'DrugId', 'EaseofUse', 'Effectiveness',
       'Reviews', 'Satisfaction', 'Sex', 'Sides', 'UsefulCount'],
      dtype='object')

###  3.2. Data Selection

Tahap kedua dalam pra-pemrosesan yaitu data selection dikarenakan memfokuskan pada atribute yang akan digunakan pada penelitian 

In [6]:
list_sufficient = [val for val, cnt in df_drugs["Drug"].value_counts().iteritems() if cnt >= 500]
df_drugs_sub = df_drugs[df_drugs["Drug"].isin(list_sufficient)]

In [7]:
df_drugs_sub = df_drugs_sub.drop(columns = ["SubName","Reviews"])
df_drugs_sub.head()

,ListActiveIngredient,ListInactiveIngredient,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Satisfaction,Sex,Sides,UsefulCount
0,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...",75 or over,Other,3/18/2019,isosorbide mononitrate,6054,1,1,1,Male,"Headache , dizziness , lightheadedness , na...",0
1,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...",55-64,Other,4/19/2018,isosorbide mononitrate,6054,1,5,1,Female,"Headache , dizziness , lightheadedness , na...",0
2,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...",75 or over,Angina Pectoris Prevention,4/12/2018,isosorbide mononitrate,6054,5,5,5,Male,"Headache , dizziness , lightheadedness , na...",4
3,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...",75 or over,Angina Pectoris Prevention,9/21/2017,isosorbide mononitrate,6054,5,5,5,Male,"Headache , dizziness , lightheadedness , na...",10
4,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...",75 or over,Angina Pectoris Prevention,8/4/2017,isosorbide mononitrate,6054,5,5,5,Male,"Headache , dizziness , lightheadedness , na...",6


###  3.3. Cleaning Data

Tahap pertama pada proses _cleaning data_ yaitu dengan mengubah nilai pada kolom `ListActiveIngredient` ke dalam bentuk _one-hot encoding vector_. _One-hot encoding_ mengubah fitur kategorikal/nominal ke format yang lebih sesuai digunakan dalam algoritma klasifikasi dan regresi. Dalam sistem rekomendasi ini, algoritma klasifikasi digunakan sebagai pembentuk model _hybrid_. 

In [8]:
def active_ingredient_text_cleaning(txt_in):
    txt_out = txt_in.replace("[[", "")
    txt_out = txt_out.replace("]]", "")
    list_out = txt_out.split("], [")
    list_out = [[j.strip("\'") for j in i.split(",")][0].lower() for i in list_out]
    return list_out

list_active_ingredients = []
for i in df_drugs_sub['ListActiveIngredient']:
    list_active_ingredients.append(active_ingredient_text_cleaning(i))

# hapus kolom list active ingredient
df_drugs_sub = df_drugs_sub.drop(columns = ["ListActiveIngredient"])
# buat kolom active ingredients berdasarkan list active ingredients
df_drugs_sub["ActiveIngredients"] = list_active_ingredients

Kemudian mengubah nilai pada kolom `ListInactiveIngredient` ke dalam bentuk _one-hot encoding vector_ sama seperti pada `ListActiveIngredient`. 

In [9]:
def inactive_ingredient_text_cleaning(txt_in):
    txt_out = txt_in.replace("[[", "")
    txt_out = txt_out.replace("]]", "")
    list_out = txt_out.split("], [")
    list_out = [[j.strip("\'") for j in i.split(",")][0].lower() for i in list_out]
    return list_out

list_inactive_ingredients = []
for i in df_drugs_sub['ListInactiveIngredient']:
    list_inactive_ingredients.append(inactive_ingredient_text_cleaning(i))

# hapus kolom list inactive ingredient
df_drugs_sub = df_drugs_sub.drop(columns = ["ListInactiveIngredient"])
# buat kolom inactive ingredients berdasarkan list inactive ingredients
df_drugs_sub["InactiveIngredients"] = list_inactive_ingredients

Langkah berikutnya yaitu mengubah nilai pada kolom `sides` ke dalam bentuk _one-hot encoding vector_, sama seperti pada `ListActiveIngredient` dan `ListInactiveIngredient`. 

In [10]:
def sides_text_cleaning(txt_in):
    # ganti whitespace lebih dari 1 menjadi 1 
    txt_out = re.sub('\s+', ' ', txt_in)
    # hapus kalimat keterangan
    txt_out = txt_out.replace("If any of these effects persist or worsen, tell your doctor or pharmacist promptly.", "")
    txt_out = txt_out.replace("If any of these effects last or get worse, tell your doctor or pharmacist promptly.", "")
    txt_out = txt_out.replace("may occur, especially during the first 2 hours after you take the medication .", "")
    txt_out = txt_out.replace("may occur as your body adjusts to this medication .", "")
    txt_out = txt_out.replace("may occur as your body adjusts to the medication .", "")
    txt_out = txt_out.replace("may also occur.", "")
    txt_out = txt_out.replace("may occur.", "")
    txt_out = txt_out.replace("and ", "")
    txt_out = txt_out.replace("your ", "")
    txt_out = txt_out.replace("or ", "")
    return txt_out

list_sides = []
for id, val in df_drugs_sub["Sides"].iteritems():
    # pembersihan text sides dan tokenisasi text sides
    list_sides.append([i.strip().lower() for i in sides_text_cleaning(val).split(",")])

# hapus kolom sides yang lama
df_drugs_sub = df_drugs_sub.drop(columns = ["Sides"])
# buat kolom sides yang baru berdasarkan list sides
df_drugs_sub["Sides"] = list_sides

###  3.4. Data Transformation

Pada `df_drugs` terdapat nilai atribut `Age` dan `Sex` yang kosong. Oleh karena itu, dilakukan imputasi dengan menggunakan Simple Imputer.

In [11]:
# label encoding map untuk nilai pada sex
map_sex = {"Male":0, "Female":1}
# label encoding map untuk nilai pada age
map_age = {'0-2':0,
           '3-6':1,
           '7-12':2,
           '13-18':3,
           '19-24':4,
           '25-34':5,
           '35-44':6,
           '45-54':7,
           '55-64':8,
           '65-74':9,
           '75 or over':10}

df_tmp = df_drugs_sub.copy()
# label encoding pada sex
df_tmp["Sexmap"] = df_tmp["Sex"].map(map_sex)
# label encoding pada age
df_tmp["Agemap"] = df_tmp["Age"].map(map_age)

# inverted label encoding map untuk nilai pada sex
inv_map_sex = {v: k for k, v in map_sex.items()}
# inverted label encoding map untuk nilai pada age
inv_map_age = {v: k for k, v in map_age.items()}

# penanganan missing values pada sex dan age
# imp = KNNImputer(n_neighbors=1)
imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
mat = imp.fit_transform(df_tmp[["Agemap", "Sexmap", "Satisfaction"]])
df_tmp2 = pd.DataFrame(mat, columns=["NewAge", "NewSex", "NewSatisfaction"])
df_drugs_sub["Sex"] = df_tmp2["NewSex"].map(inv_map_sex).tolist()
df_drugs_sub["Age"] = df_tmp2["NewAge"].map(inv_map_age).tolist()

# reset index setiap baris
df_drugs_sub = df_drugs_sub.reset_index(drop=True)

## 4. Pembentukan Set Data Obat dan Set Data Rating

Untuk menerapkan _item-based collaborative filtering_ pada sistem rekomendasi obat, dibutuhkan matriks pengguna/obat. Pada matriks tersebut terdapat _"user id"_, _"drug id"_, dan _"drug rating"_. Ilustrasi matriks tersebut adalah sebagai berikut.

|  | drug<sub>1</sub> | drug<sub>2</sub> | drug<sub>3</sub> |
| :---: | :---: | :---: | :---: |
| user<sub>1</sub> | rating<sub>user<sub>1</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>3</sub></sub> |
| user<sub>2</sub> | rating<sub>user<sub>2</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>3</sub></sub> |
| user<sub>3</sub> | rating<sub>user<sub>3</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>3</sub></sub> |
| user<sub>4</sub> | rating<sub>user<sub>4</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>3</sub></sub> |
| user<sub>5</sub> | rating<sub>user<sub>5</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>3</sub></sub> |


_"user id"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, user id diambil dari df_user

_"drug id"_ tersedia dalam `df_drugs`. Oleh karena itu, drug id akan diambil dari df_item

_"drug rating"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, drug rating diambil dari df_rating

In [12]:
# Data Drug
df_item = pd.DataFrame({'Drug':df_drugs_sub['Drug'],
                        'ActiveIngredients':df_drugs_sub['ActiveIngredients'],
                        'InactiveIngredients':df_drugs_sub['InactiveIngredients'],
                        'Sides':df_drugs_sub['Sides']})

# buat duplikat df_item, yaitu df_tmp
df_tmp = df_item.copy()
# ubah nilai ActiveIngredients dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp['ActiveIngredients'] = [" ".join(i) for i in df_tmp['ActiveIngredients']]
# ubah nilai InactiveIngredients dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp['InactiveIngredients'] = [" ".join(i) for i in df_tmp['InactiveIngredients']]
# ubah nilai Sides dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp['Sides'] = [" ".join(i) for i in df_tmp['Sides']]
# hapus baris data duplikat pada df_tmp dan simpan index
# baris data yang dipertahankan dalam list_index
list_index = list(df_tmp.drop_duplicates().index)
# gunakan list_index yang didapatkan untuk memfilter
# baris data pada df_item
df_item = df_item.iloc[list_index]
# reset index setiap baris
df_item = df_item.reset_index(drop=True)
# buat kolom ItemId berdasarkan index setiap baris
df_item["ItemId"] = df_item.index + 1
# rapikan posisi kolom dengan memindahkan 
# posisi kolom "ItemId" ke kolom pertama
df_item = df_item[['ItemId','Drug','ActiveIngredients','InactiveIngredients','Sides']]
df_item.head()

,ItemId,Drug,ActiveIngredients,InactiveIngredients,Sides
0,1,isosorbide mononitrate,[isosorbide mononitrate],"[lactose, talc, silicon dioxide, cellulose, al...","[headache, dizziness, lightheadedness, nausea,..."
1,2,isosorbide mononitrate,[isosorbide mononitrate],"[silicon dioxide, hydrogenated castor oil, hyp...","[headache, dizziness, lightheadedness, nausea,..."
2,3,isosorbide mononitrate,[isosorbide mononitrate],[[]],"[headache, dizziness, lightheadedness, nausea,..."
3,4,isosorbide mononitrate,[isosorbide mononitrate],"[silicon dioxide, hydrogenated castor oil, hyp...","[headache, dizziness, lightheadedness, nausea,..."
4,5,prednisone,[prednisone],"[lactose monohydrate, magnesium stearate, micr...","[nausea, vomiting, loss of appetite, heartburn..."


In [13]:
# Data User
df_user = pd.DataFrame({'Age':df_drugs_sub['Age'],
                        'Sex':df_drugs_sub['Sex'],
                        'Condition':df_drugs_sub['Condition']})

# buat kolom UserId berdasarkan index setiap baris
df_user["UserId"] = df_user.index + 1
# rapikan posisi kolom dengan memindahkan 
# posisi kolom "UserId" ke kolom pertama
df_user = df_user[['UserId','Age','Sex','Condition']]
df_user.head()

,UserId,Age,Sex,Condition
0,1,75 or over,Male,Other
1,2,55-64,Female,Other
2,3,75 or over,Male,Angina Pectoris Prevention
3,4,75 or over,Male,Angina Pectoris Prevention
4,5,75 or over,Male,Angina Pectoris Prevention


In [14]:
# Data Rating
df_tmp2 = df_drugs_sub.copy()
# ubah nilai ActiveIngredients dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp2['ActiveIngredients'] = [" ".join(i) for i in df_tmp2['ActiveIngredients']]
# ubah nilai InactiveIngredients dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp2['InactiveIngredients'] = [" ".join(i) for i in df_tmp2['InactiveIngredients']]
# ubah nilai Sides dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp2['Sides'] = [" ".join(i) for i in df_tmp2['Sides']]

df_tmp = df_item.copy()
# ubah nilai ActiveIngredients dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp['ActiveIngredients'] = [" ".join(i) for i in df_tmp['ActiveIngredients']]
# ubah nilai InactiveIngredients dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp['InactiveIngredients'] = [" ".join(i) for i in df_tmp['InactiveIngredients']]
# ubah nilai Sides dari list of string menjadi
# string untuk memungkinkan fungsi drop_duplicates dijalankan
df_tmp['Sides'] = [" ".join(i) for i in df_tmp['Sides']]

df_rating = df_tmp2.merge(df_tmp,
                          how="left",
                          on=["Drug", "ActiveIngredients", "InactiveIngredients", "Sides"])
df_rating["UserId"] = df_user["UserId"]
df_rating = df_rating[['UserId',
                       'ItemId',
                       'EaseofUse',
                       'Effectiveness',
                       'Satisfaction']]
df_rating.head()

,UserId,ItemId,EaseofUse,Effectiveness,Satisfaction
0,1,1,1,1,1
1,2,1,1,5,1
2,3,1,5,5,5
3,4,1,5,5,5
4,5,1,5,5,5


In [15]:
# one hot encoding pada kolom Condition
one_hot = pd.get_dummies(df_user['Condition'])
# hapus kolom Condition karena saat ini sudah di-encode
df_user = df_user.drop('Condition', axis = 1)
# gabungkan hasil encode ke df_user
df_user = df_user.join(one_hot)

# one hot encoding pada kolom Sex
one_hot = pd.get_dummies(df_user['Sex'])
# hapus kolom Sex karena saat ini sudah di-encode
df_user = df_user.drop('Sex', axis = 1)
# gabungkan hasil encode ke df_user
df_user = df_user.join(one_hot)

# ordinal label encoding map untuk nilai pada Age
map_age = {'0-2':0,
           '3-6':1,
           '7-12':2,
           '13-18':3,
           '19-24':4,
           '25-34':5,
           '35-44':6,
           '45-54':7,
           '55-64':8,
           '65-74':9,
           '75 or over':10}
df_tmp = df_user.copy()
# label encoding pada Age
df_user["Age"] = df_tmp["Age"].map(map_age)

In [16]:
# one-hot encoding pada kolom ActiveIngredients
mlb = MultiLabelBinarizer()
df_item = df_item.join(pd.DataFrame(mlb.fit_transform(df_item.pop("ActiveIngredients")),
                                    columns=mlb.classes_,
                                    index=df_item.index))

# one-hot encoding pada kolom InactiveIngredients
mlb = MultiLabelBinarizer()
df_item = df_item.join(pd.DataFrame(mlb.fit_transform(df_item.pop("InactiveIngredients")),
                                    columns=mlb.classes_,
                                    index=df_item.index))

# one-hot encoding pada kolom Sides
mlb = MultiLabelBinarizer()
df_item = df_item.join(pd.DataFrame(mlb.fit_transform(df_item.pop("Sides")),
                                    columns=mlb.classes_,
                                    index=df_item.index))

# hapus kolom Drug (id=1), [] (id=23), " " (id=100)
df_item = df_item.drop(df_item.iloc[:, [1, 23, 100]], axis=1)

## 5. Pembangunan Model Rekomendasi

Pada tahap pembangunan model rekomendasi, data dibagi menjadi 70% data latih dan sisanya 30% data uji.

In [17]:
# 70% baris data dipilih secara random 
# sebagai data train dan sisanya sebagai data test
df_rating_train = df_rating.sample(frac = 0.7)
df_rating_train = df_rating_train.reset_index(drop=True)
df_rating_test = df_rating.drop(df_rating_train.index)
df_rating_test = df_rating_test.reset_index(drop=True)

In [18]:
x_train = df_rating_train[['UserId', 'ItemId']]
y_train = df_rating_train[['Satisfaction']]
x_test = df_rating_test[['UserId', 'ItemId']]
y_test = df_rating_test['Satisfaction']

In [19]:
# merge df_item dan df_user untuk masing-masing set data latih dan uji
x_train = x_train.join(df_user.set_index('UserId'),
                       on = 'UserId').join(df_item.set_index('ItemId'),
                                           on = 'ItemId')

x_test = x_test.join(df_user.set_index('UserId'),
                     on = 'UserId').join(df_item.set_index('ItemId'),
                                         on = 'ItemId')

x_train digabung untuk mengambil semua atribute pada dataframe user dan dataframe item dengan menggunakan UserId dan ItemId

### 5.1. Optimasi Hiperparameter

Pada bagian ini diterapkan **Randomized Search** untuk mencari kombinasi nilai hiperparameter yang mengoptimalkan performa model berdasarkan set data yang tersedia.

#### 5.1.1. Random Forest

In [20]:
# total pohon dalam random forest
n_estimators = [int(x) for x in np.linspace(start = 200,
                                            stop = 2000,
                                            num = 10)]

# total fitur yang dipertimbangkan pada setiap pemisahan 
max_features = ['auto', 'sqrt']

# jumlah total maksimum level di setiap pohon keputusan dengan mengembalikan 
# angka dengan interval 10, 110
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# jumlah minimal dari titik sampel yang ada pada node sebelum dibagi
min_samples_split = [2, 5, 10]

# titik sampel minimal yang diperbolehkan pada node leaf
min_samples_leaf = [1, 2, 4]

# mengambil titik sampel
bootstrap = [True, False]

# ruang pencarian dalam bentuk hyperparameter grid
search_space_rf = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

In [21]:
# menggunakan random grid untuk mencari hyperparameter terbaik
# Pertama buat model awal
rf = RandomForestClassifier()

# Mencari paramater secara acak, menggunakan cross validation 3 kali lipat
# Mencari kombinasi yang berbeda pada 100 kombinasi yang berbeda
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf,
                               param_distributions = search_space_rf,
                               n_iter = 1,
                               cv = 3,
                               verbose = 0,
                               random_state = 42,
                               n_jobs = -1)

# fit the random search model
rf_random.fit(x_train.iloc[:, 2:x_train.shape[1]].values,
              y_train["Satisfaction"].values)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=1,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42)

#### 5.1.2. k-Nearest Neighbors

In [22]:
# Rentang nilai algorithm
algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']

# Rentang Nilai n_neighbors
n_neighbors = [5, 7, 9, 11]

# rentang nilai weight
weights = ['uniform', 'distance']

# rentang nilai leaf size
leaf_size = [10, 20, 30]

# ruang pencarian dalam bentuk hyperparameter grid
search_space_kn = {'n_neighbors': n_neighbors,
                   'weights': weights,
                   'algorithm': algorithms,
                   'leaf_size': leaf_size}

In [23]:
# menggunakan random grid untuk mencari hyperparameter terbaik
# Pertama buat model awal
kn = KNeighborsClassifier(metric='cosine')

# Mencari paramater secara acak, menggunakan cross validation 3 kali lipat
# Mencari kombinasi yang berbeda pada 100 kombinasi yang berbeda
# search across 100 different combinations, and use all available cores
kn_random = RandomizedSearchCV(estimator = kn,
                               param_distributions = search_space_kn,
                               n_iter = 1,
                               cv = 3,
                               verbose = 0,
                               random_state = 42,
                               n_jobs = -1)

# fit the random search model
kn_random.fit(x_train.iloc[:, 2:x_train.shape[1]].values, 
              y_train["Satisfaction"].values)

RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(metric='cosine'),
                   n_iter=1, n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': [10, 20, 30],
                                        'n_neighbors': [5, 7, 9, 11],
                                        'weights': ['uniform', 'distance']},
                   random_state=42)

### 5.2. Pembangunan Model Rekomendasi

#### Content-based Filtering based on Random Forest

In [24]:
# model 1
model1 = RandomForestClassifier()
model1.set_params(**rf_random.best_params_)
model1.fit(x_train.iloc[:, 2:x_train.shape[1]].values, 
           y_train["Satisfaction"].values)

RandomForestClassifier(max_depth=50, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=10, n_estimators=200)

Model1 rekomendasi yang digunakan untuk content-based filtering adalah RandomForestClasifier

#### Collaborative-based Filtering based on k-Nearest Neighbors

### -------------------------------------------------------- update --------------------------------------------------------

In [25]:
n_users = df_rating_train.UserId.unique().shape[0]
n_items = df_rating_train.ItemId.unique().shape[0]
n_users = df_rating_train['UserId'].max()
n_items = df_rating_train['ItemId'].max()
A = np.zeros((n_users,n_items))
for line in df_rating_train.itertuples():
    A[line[1]-1,line[2]-1] = line[3]
print("Original rating matrix : ",A)

Original rating matrix :  [[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 5.]
 [0. 0. 0. ... 0. 0. 4.]
 [0. 0. 0. ... 0. 0. 4.]]


In [26]:
for i in range(len(A)):
  for j in range(len(A[0])):
    if A[i][j]>=3:
      A[i][j]=1
    else:
      A[i][j]=0

In [27]:
csr_sample = csr_matrix(A)
print(csr_sample)

  (2, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (15, 0)	1.0
  (16, 0)	1.0
  (19, 0)	1.0
  (22, 0)	1.0
  (24, 0)	1.0
  (25, 0)	1.0
  (29, 0)	1.0
  (30, 0)	1.0
  (32, 0)	1.0
  (33, 0)	1.0
  (35, 0)	1.0
  (37, 0)	1.0
  (38, 0)	1.0
  (41, 0)	1.0
  (42, 0)	1.0
  (46, 0)	1.0
  (47, 0)	1.0
  (49, 0)	1.0
  :	:
  (55241, 42)	1.0
  (55245, 42)	1.0
  (55248, 42)	1.0
  (55250, 42)	1.0
  (55253, 42)	1.0
  (55254, 42)	1.0
  (55255, 42)	1.0
  (55256, 42)	1.0
  (55260, 42)	1.0
  (55262, 42)	1.0
  (55263, 42)	1.0
  (55265, 42)	1.0
  (55266, 42)	1.0
  (55268, 42)	1.0
  (55269, 42)	1.0
  (55270, 42)	1.0
  (55273, 42)	1.0
  (55275, 42)	1.0
  (55277, 42)	1.0
  (55278, 42)	1.0
  (55279, 42)	1.0
  (55280, 42)	1.0
  (55282, 42)	1.0
  (55283, 42)	1.0
  (55284, 42)	1.0


In [28]:
# model 2
model2 = KNeighborsClassifier(metric='cosine')
model2.set_params(**kn_random.best_params_)
model2.fit(csr_sample)
# model2.fit(x_train.iloc[:, 2:x_train.shape[1]].values, 
#            y_train["Satisfaction"].values)

TypeError: fit() missing 1 required positional argument: 'y'

### -------------------------------------------------------- update --------------------------------------------------------

Model2 rekomendasi yang digunakan untuk Collaborative-based Filtering based on k-Nearest Neigboard adalah KNeighborsClassifier

## 6. Pengujian Model Rekomendasi

Pengujian model rekomendasi dilakukan menggunakan evaluasi metrik rekomendasi RMSE _(Root Mean Square Error)_

#### Content-based Filtering based on Random Forest

In [ ]:
pred1 = model1.predict(x_test.iloc[:, 2:x_test.shape[1]].values)
rmse = np.sqrt(np.mean((pred1 - y_test.to_numpy())**2))
print("content-based rmse = %.4f" %rmse)

#### Collaborative-based Filtering based on k-Nearest Neighbors

In [ ]:
pred2 = model2.predict(x_test.iloc[:, 2:x_test.shape[1]].values)
rmse = np.sqrt(np.mean((pred2 - y_test.to_numpy())**2))
print("collaborative-based rmse = %.4f" %rmse)

In [ ]:
# print("Satisfaction Aktual")
# print([i for i in y_test])

In [ ]:
# print("Satisfaction Prediction dari Random Forest")
# print([i for i in pred1])

In [ ]:
# print("Satisfaction Prediction dari k-Nearest Neighbors")
# print([i for i in pred2])

#### Hybrid

In [ ]:
# simple average
bobot = 0.5
pred3 = pred1*bobot + pred2*(1-bobot)
rmse = np.sqrt(np.mean((pred3 - y_test.to_numpy())**2))
print("hybrid rmse = %.4f" %rmse)

In [ ]:
# optimal weight
bobot = 0.468749004
pred3 = pred1*bobot + pred2*(1-bobot)
rmse = np.sqrt(np.mean((pred3 - y_test.to_numpy())**2))
print("hybrid rmse = %.4f" %rmse)

In [ ]:
# optimal weight ignoring error correlation
bobot = 0.03085328059
pred3 = pred1*bobot + pred2*(1-bobot)
rmse = np.sqrt(np.mean((pred3 - y_test.to_numpy())**2))
print("hybrid rmse = %.4f" %rmse)

Observasi rentang konfigurasi pembobotan yang optimal. Dalam hal ini digunakan fungsi `linspace` pada NumPy untuk membuat sekuens _weight_.

In [ ]:
chart_val = []

w = np.linspace(0,1,21)

for i in w:
    pred3 = pred1*i + pred2*(1-i)
    rmse = np.sqrt(np.mean((pred3 - y_test.to_numpy())**2))
    chart_val.append([i, rmse])

chart_val_np = np.array(chart_val)
plt.plot(chart_val_np[:, 0], chart_val_np[:,1])
plt.xlabel('Random Forest Model Weight Proportional to Item Based Collaborative')
plt.ylabel('RMSE')

In [ ]:
#chart_val_np

Dapat dilihat bahwa _weight_ terbaik (RMSE terkecil) adalah di antara `0,4` s.d. `0.6`.

---